<a href="https://colab.research.google.com/github/aicrashcoursewinter24/tuan-le-langchain/blob/agents-pr/agents_without_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install langchain-openai
!pip install openai
!pip install -U duckduckgo-search
!pip install langchainhub
!pip install -U langchain-community tavily-python
!pip install faiss-gpu

  Using cached tiktoken-0.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.5.1
    Uninstalling tiktoken-0.5.1:
      Successfully uninstalled tiktoken-0.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tavily-python 0.3.0 requires tiktoken==0.5.1, but you have tiktoken 0.5.2 which is incompatible.
  Using cached tiktoken-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.5.2
    Uninstalling tiktoken-0.5.2:
      Successfully uninstalled tiktoken-0.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

In [2]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
search = TavilySearchResults()

In [5]:
search.invoke("what is the most uptrending stock today?")

[{'url': 'https://stock-screener.org/trending-stocks.aspx',
  'content': 'Trending Stocks Today  Trending Stocks  the stock is in an uptrend.  stocks with our trending stock scanner today.Last Updated Jan 12, 2024 To get intraday penny stocks update (every 5-10 minutes), please signup for a free account. What Are Trending Stocks? Trending stocks are stocks that are trending up or on the move either today or recently.'},
 {'url': 'https://www.marketbeat.com/market-data/uptrend-stocks/',
  'content': "Top Uptrending Stocks  Are Uptrending Stocks a Good Investment?  How to Invest in Uptrending Stocks  Sign up for MarketBeat All Access to gain access to MarketBeat's full suite of research tools: Uptrend StocksUptrend Stock List - January 2024 Uptrend Stocks Uptrend stocks are public companies that have been increasing in price for five consecutive days. Investors may use this report to identify stocks that are increasing in price in the near term of are recovering from a previous pull back

In [6]:
import os
from langchain.agents import AgentExecutor, AgentType, load_tools, initialize_agent
from langchain.chat_models import ChatOpenAI

In [7]:
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


In [8]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [10]:
retriever.get_relevant_documents("how to upload a dataset")[0]

Document(page_content="dataset uploading.Once we have a dataset, how can we use it to test changes to a prompt or chain? The most basic approach is to run the chain over the data points and visualize the outputs. Despite technological advancements, there still is no substitute for looking at outputs by eye. Currently, running the chain over the data points needs to be done client-side. The LangSmith client makes it easy to pull down a dataset and then run a chain over them, logging the results to a new project associated with the dataset. From there, you can review them. We've made it easy to assign feedback to runs and mark them as correct or incorrect directly in the web app, displaying aggregate statistics for each test project.We also make it easier to evaluate these runs. To that end, we've added a set of evaluators to the open-source LangChain library. These evaluators can be specified when initiating a test run and will evaluate the results once the test run completes. If we‚Äôr

In [11]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [12]:
tools = [search, retriever_tool]

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [14]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [18]:
agent_executor.invoke({"input": "what is an API key?"})



> Entering new AgentExecutor chain...
An API key, or Application Programming Interface key, is a unique code that is used to authenticate and identify the user or application making requests to an API (Application Programming Interface). APIs are used to allow different software applications to communicate and interact with each other. 

API keys are typically used to track and control access to an API. When making a request to an API, the API key is included in the request to authenticate the user or application. This helps the API provider to monitor and manage the usage of their API, as well as enforce any usage limits or restrictions.

API keys are often required by developers when integrating third-party APIs into their applications. They are usually provided by the API provider and should be kept confidential, as they grant access to the API and its associated resources.

> Finished chain.


{'input': 'what is an API key?',
 'output': 'An API key, or Application Programming Interface key, is a unique code that is used to authenticate and identify the user or application making requests to an API (Application Programming Interface). APIs are used to allow different software applications to communicate and interact with each other. \n\nAPI keys are typically used to track and control access to an API. When making a request to an API, the API key is included in the request to authenticate the user or application. This helps the API provider to monitor and manage the usage of their API, as well as enforce any usage limits or restrictions.\n\nAPI keys are often required by developers when integrating third-party APIs into their applications. They are usually provided by the API provider and should be kept confidential, as they grant access to the API and its associated resources.'}

In [19]:
agent_executor.invoke({"input": "how to create an ASP.NET Core environment in VSCode?"})



> Entering new AgentExecutor chain...
To create an ASP.NET Core environment in VSCode, you can follow these steps:

1. Install the necessary software:
   - Install .NET Core SDK: Visit the official .NET Core website (https://dotnet.microsoft.com/download) and download the latest version of the .NET Core SDK for your operating system. Follow the installation instructions to complete the installation.
   - Install Visual Studio Code: Visit the official Visual Studio Code website (https://code.visualstudio.com/) and download the appropriate version for your operating system. Follow the installation instructions to complete the installation.

2. Open Visual Studio Code and install the necessary extensions:
   - Open Visual Studio Code.
   - Click on the Extensions icon in the left sidebar (or press `Ctrl+Shift+X`).
   - Search for and install the "C#" extension by Microsoft.

3. Create a new ASP.NET Core project:
   - Open the Command Palette by pressing `Ctrl+Shift+P`.
   - Type "dotnet

{'input': 'how to create an ASP.NET Core environment in VSCode?',
 'output': 'To create an ASP.NET Core environment in VSCode, you can follow these steps:\n\n1. Install the necessary software:\n   - Install .NET Core SDK: Visit the official .NET Core website (https://dotnet.microsoft.com/download) and download the latest version of the .NET Core SDK for your operating system. Follow the installation instructions to complete the installation.\n   - Install Visual Studio Code: Visit the official Visual Studio Code website (https://code.visualstudio.com/) and download the appropriate version for your operating system. Follow the installation instructions to complete the installation.\n\n2. Open Visual Studio Code and install the necessary extensions:\n   - Open Visual Studio Code.\n   - Click on the Extensions icon in the left sidebar (or press `Ctrl+Shift+X`).\n   - Search for and install the "C#" extension by Microsoft.\n\n3. Create a new ASP.NET Core project:\n   - Open the Command Pal

In [20]:
agent_executor.invoke({"input": "what is the most popular programming language in 2023?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'most popular programming language in 2023'}`


[{'url': 'https://www.stackscale.com/blog/most-popular-programming-languages/', 'content': "Most popular programming languages in 2023  Top 10 programming languages in 2023  Let’s take a closer look at the most used programming languages in 2023. Python  popular programming languages in 2022. But, according to Stack Overflow’s 2022 Developer Survey, JavaScript, HTML/CSSWhich are the most popular programming languages in 2023? There are different lists of the most commonly used programming languages: the PYPL ranking (monthly updated) and the list of Stack Overflow's Developer Survey 2023. Table of contents 1 Top 10 programming languages in 2023 2 Programming languages ranking 2023 2.1 Python 2.2 JavaScript (JS)"}, {'url': 'https://www.techrepublic.com/article/tiobe-index-language-rankings/', 'content': "Top 10 programming languages in May 2023 

{'input': 'what is the most popular programming language in 2023?',
 'output': 'According to the search results, the most popular programming language in 2023 is C#.'}

In [21]:
def build_simple_agent():
  llm = ChatOpenAI(openai_api_base="https://api.openai.com/v1",
                   api_key=OPENAI_API_KEY,
                   model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
                   temperature=0.7,
                   verbose=True)
  tools = load_tools(["llm-math", "ddg-search"], llm=llm)
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
  print(agent.agent.llm_chain.prompt.template)
  return agent

In [22]:
def chat_with_agent(agent: AgentExecutor, user_input: str):
  output = agent.invoke({"input": user_input})
  return output

In [23]:
agent = build_simple_agent()
agent.invoke({"input": "what is an API key?"})

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, duckduckgo_search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


BadRequestError: Error code: 400 - {'error': {'message': 'invalid model ID', 'type': 'invalid_request_error', 'param': None, 'code': None}}